In [9]:
import os
from collections import defaultdict

import numpy as np
import optuna
from optuna.integration.mlflow import MLflowCallback
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                            precision_score, recall_score, 
                            f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import mlflow

In [19]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os

# Environment setup
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)

    metrics = defaultdict(list)
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, probas)
        
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)

    err_1 = median(array(metrics['err1']))
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))

    trial.set_user_attr("err1", err_1)
    trial.set_user_attr("err2", err_2)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("logloss", logloss)

    return auc

# Clean up any active runs
mlflow.end_run()

# Setup experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Create parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback with proper parent run context
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Log best results
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params).fit(X_train, y_train)
    
    # Explicitly log all parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metric("auc", study.best_value)
    mlflow.sklearn.log_model(best_model, "model")
    
    # Ensure run is marked as FINISHED
    mlflow.end_run()

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

/tmp/ipykernel_82920/1750834979.py:103: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:19:51,421] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:19:52,634] Trial 4 finished with value: 0.8071864451360531 and parameters: {'learning_rate': 0.002452135027731656, 'depth': 3, 'l2_leaf_reg': 3.447183000891331, 'random_strength': 3.7695723293036867}. Best is trial 3 with value: 0.8219169519762917.
[I 2025-08-12 23:19:54,106] Trial 5 finished with value: 0.8117012864665529 and parameters: {'learning_rate': 0.0013521895172016907, 'depth': 7, 'l2_leaf_reg': 1.781489159094527, 'random_strength': 3.948712745192102}. Best is trial 3 with value: 0.8219169519762917.
[I 2025-08-12 23:19:55,024] Trial 6 finished with value: 0.8122915360865393 and parameters: {'learning_rate': 0.010095223901010765, 'depth': 2, 'l2_leaf_reg': 0.9863798071489918,

0:	learn: 0.6774026	total: 1.61ms	remaining: 1.61s
1:	learn: 0.6625839	total: 3.14ms	remaining: 1.57s
2:	learn: 0.6489590	total: 4.65ms	remaining: 1.55s
3:	learn: 0.6359476	total: 6.17ms	remaining: 1.54s
4:	learn: 0.6251907	total: 7.67ms	remaining: 1.53s
5:	learn: 0.6151933	total: 9.15ms	remaining: 1.52s
6:	learn: 0.6060980	total: 10.6ms	remaining: 1.51s
7:	learn: 0.5963681	total: 12.1ms	remaining: 1.5s
8:	learn: 0.5871717	total: 13.6ms	remaining: 1.49s
9:	learn: 0.5783969	total: 15ms	remaining: 1.49s
10:	learn: 0.5704020	total: 16.5ms	remaining: 1.48s
11:	learn: 0.5634128	total: 18.1ms	remaining: 1.49s
12:	learn: 0.5568929	total: 19.6ms	remaining: 1.49s
13:	learn: 0.5499787	total: 21ms	remaining: 1.48s
14:	learn: 0.5437712	total: 22.5ms	remaining: 1.48s
15:	learn: 0.5377809	total: 24ms	remaining: 1.48s
16:	learn: 0.5320960	total: 25.5ms	remaining: 1.48s
17:	learn: 0.5268556	total: 27ms	remaining: 1.47s
18:	learn: 0.5226306	total: 28.5ms	remaining: 1.47s
19:	learn: 0.5182293	total: 30m

In [20]:
run_id

'70a6759736164afaa06ffdbe8268fa76'

In [6]:

import os
import mlflow
import boto3
import numpy as np
from dotenv import load_dotenv
from botocore.exceptions import ClientError
import psycopg
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, log_loss
from mlflow.models.signature import infer_signature

# Constants
TABLE_NAME = "users_churn"
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
EXPERIMENT_NAME = "churn_ivan_panchenko"
RUN_NAME = "feature_selection"
REGISTRY_MODEL_NAME = "churn_model_ivan_panchenko"
FS_ASSETS = "fs_assets"

# Load environment variables
load_dotenv()

def get_env_variable(var_name):
    value = os.getenv(var_name)
    if not value:
        raise ValueError(f"Environment variable {var_name} is not set in the .env file")
    return value

# Get environment variables
S3_ENDPOINT_URL = get_env_variable('S3_ENDPOINT_URL')
S3_BUCKET_NAME = get_env_variable('S3_BUCKET_NAME')
AWS_ACCESS_KEY_ID = get_env_variable('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = get_env_variable('AWS_SECRET_ACCESS_KEY')

# Database connection setup
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv('DB_DESTINATION_HOST'),
    "port": os.getenv('DB_DESTINATION_PORT'),
    "dbname": os.getenv('DB_DESTINATION_NAME'),
    "user": os.getenv('DB_DESTINATION_USER'),
    "password": os.getenv('DB_DESTINATION_PASSWORD')
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])
connection.update(postgres_credentials)

# Fetch data from database
with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
print(f"Table size: {df.shape[0]} rows; {df.shape[1]} columns")

# Define features and target
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"
test_size = 0.2

# Check if 'split' column exists, if not, create a random split
if 'split' not in df.columns:
    print("'split' column not found. Creating a random split.")
    df['split'] = np.random.choice(['train', 'test'], size=len(df), p=[1-test_size, test_size])

# Split the data
df_train = df[df['split'] == 'train']
df_test = df[df['split'] == 'test']

X_train = df_train[features]
y_train = df_train[target]
X_test = df_test[features]
y_test = df_test[target]

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")


Table size: 7043 rows; 22 columns
'split' column not found. Creating a random split.
Training set size: (5643, 3)
Test set size: (1400, 3)


In [25]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os

# Environment setup
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)

    metrics = defaultdict(list)
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, probas)
        
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)

    err_1 = median(array(metrics['err1']))
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))

    trial.set_user_attr("err1", err_1)
    trial.set_user_attr("err2", err_2)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("logloss", logloss)

    return auc

# Clean up any active runs
mlflow.end_run()

# Setup experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Create parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback with proper parent run context
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Train final model with best params
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Log model with all required artifacts
    mlflow.catboost.log_model(
        cb_model=best_model,
        artifact_path="model",
        registered_model_name="best_catboost_model",
        conda_env={
        #     'channels': ['defaults'],
            'dependencies': []
        #         'python=3.8.5',
        #         'pip': {
        #             'pip': [
        #                 'catboost==1.0.6',
        #                 'mlflow'
        #             ]
        #         }
        #     ],
        #     'name': 'catboost_env'
        }
    )
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

/tmp/ipykernel_82920/2709353858.py:103: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:25:02,895] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:25:04,185] Trial 24 finished with value: 0.8249100144140449 and parameters: {'learning_rate': 0.03967371390005438, 'depth': 2, 'l2_leaf_reg': 4.3710749504811055, 'random_strength': 0.6947147550035017}. Best is trial 24 with value: 0.8249100144140449.
[I 2025-08-12 23:25:06,337] Trial 25 finished with value: 0.8247567471025904 and parameters: {'learning_rate': 0.0599719269351081, 'depth': 2, 'l2_leaf_reg': 4.366907966343082, 'random_strength': 1.3773568079745306}. Best is trial 24 with value: 0.8249100144140449.
[I 2025-08-12 23:25:08,159] Trial 26 finished with value: 0.8241936093672869 and parameters: {'learning_rate': 0.026040047944193817, 'depth': 2, 'l2_leaf_reg': 4.45732599451463

0:	learn: 0.6777433	total: 1.47ms	remaining: 1.46s
1:	learn: 0.6622632	total: 2.78ms	remaining: 1.39s
2:	learn: 0.6476112	total: 4.11ms	remaining: 1.36s
3:	learn: 0.6358077	total: 5.38ms	remaining: 1.34s
4:	learn: 0.6224258	total: 6.67ms	remaining: 1.33s
5:	learn: 0.6096994	total: 8ms	remaining: 1.32s
6:	learn: 0.5981318	total: 9.33ms	remaining: 1.32s
7:	learn: 0.5886444	total: 10.6ms	remaining: 1.32s
8:	learn: 0.5793530	total: 12ms	remaining: 1.32s
9:	learn: 0.5700201	total: 13.3ms	remaining: 1.32s
10:	learn: 0.5616941	total: 14.7ms	remaining: 1.32s
11:	learn: 0.5538408	total: 16.1ms	remaining: 1.32s
12:	learn: 0.5466314	total: 17.4ms	remaining: 1.32s
13:	learn: 0.5407849	total: 18.7ms	remaining: 1.32s
14:	learn: 0.5351250	total: 20.1ms	remaining: 1.32s
15:	learn: 0.5290388	total: 21.5ms	remaining: 1.32s
16:	learn: 0.5232747	total: 22.8ms	remaining: 1.32s
17:	learn: 0.5190109	total: 24.2ms	remaining: 1.32s
18:	learn: 0.5144862	total: 25.6ms	remaining: 1.32s
19:	learn: 0.5107496	total:

Successfully registered model 'best_catboost_model'.
2025/08/12 23:25:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_catboost_model, version 1
Created version '1' of model 'best_catboost_model'.


Number of finished trials: 34
Best params: {'learning_rate': 0.034908353191037286, 'depth': 3, 'l2_leaf_reg': 3.7422961704293383, 'random_strength': 0.7229092057334334}


In [27]:
run_id

'de02255a53c242c19c0af7ba92e0fbf1'

In [29]:

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Train and save final model properly
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Create proper conda environment specification
    conda_env = {
        'channels': ['defaults', 'conda-forge'],
        'dependencies': [
            
        ],
        'name': 'catboost_env'
    }
    
    # Save model with all required files
    mlflow.catboost.log_model(
        cb_model=best_model,
        artifact_path="model",
        registered_model_name="best_catboost_model",
        conda_env=conda_env
    )
    
    # Also save as sklearn flavor for compatibility
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="sklearn_model",
        conda_env=conda_env
    )
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

    # Explicitly end run to ensure all artifacts are flushed
    mlflow.end_run()

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

/tmp/ipykernel_82920/219182725.py:5: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:33:37,199] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:33:38,159] Trial 44 finished with value: 0.8248608600888045 and parameters: {'learning_rate': 0.04155502250199034, 'depth': 3, 'l2_leaf_reg': 3.697834407969645, 'random_strength': 1.293502382045665}. Best is trial 32 with value: 0.8252752692495922.
[I 2025-08-12 23:33:39,452] Trial 45 finished with value: 0.8253653715190726 and parameters: {'learning_rate': 0.044464712916537065, 'depth': 2, 'l2_leaf_reg': 3.2822864746087506, 'random_strength': 1.3708751694533676}. Best is trial 45 with value: 0.8253653715190726.
[I 2025-08-12 23:33:40,358] Trial 46 finished with value: 0.8214914446442497 and parameters: {'learning_rate': 0.04755493584783526, 'depth': 1, 'l2_leaf_reg': 3.22722068632405, '

0:	learn: 0.6810724	total: 1.32ms	remaining: 1.32s
1:	learn: 0.6672539	total: 2.86ms	remaining: 1.43s
2:	learn: 0.6507656	total: 4.47ms	remaining: 1.48s
3:	learn: 0.6361022	total: 5.62ms	remaining: 1.4s
4:	learn: 0.6261346	total: 6.73ms	remaining: 1.34s
5:	learn: 0.6130175	total: 7.88ms	remaining: 1.31s
6:	learn: 0.6014859	total: 9.11ms	remaining: 1.29s
7:	learn: 0.5903591	total: 10.3ms	remaining: 1.28s
8:	learn: 0.5831419	total: 11.7ms	remaining: 1.29s
9:	learn: 0.5768986	total: 12.9ms	remaining: 1.28s
10:	learn: 0.5709949	total: 14.2ms	remaining: 1.27s
11:	learn: 0.5653910	total: 15.4ms	remaining: 1.27s
12:	learn: 0.5576505	total: 16.6ms	remaining: 1.26s
13:	learn: 0.5528456	total: 18.5ms	remaining: 1.3s
14:	learn: 0.5458502	total: 20ms	remaining: 1.31s
15:	learn: 0.5396450	total: 21.3ms	remaining: 1.31s
16:	learn: 0.5344578	total: 22.6ms	remaining: 1.3s
17:	learn: 0.5285119	total: 23.8ms	remaining: 1.3s
18:	learn: 0.5240465	total: 25.1ms	remaining: 1.29s
19:	learn: 0.5186796	total: 

Registered model 'best_catboost_model' already exists. Creating a new version of this model...
2025/08/12 23:33:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_catboost_model, version 2
Created version '2' of model 'best_catboost_model'.


Number of finished trials: 54
Best params: {'learning_rate': 0.044464712916537065, 'depth': 2, 'l2_leaf_reg': 3.2822864746087506, 'random_strength': 1.3708751694533676}


In [30]:
run_id

'44ab60d527dc4724843066019c8237f6'

In [31]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os
import tempfile

# Environment setup
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)

    metrics = defaultdict(list)
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, probas)
        
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)

    err_1 = median(array(metrics['err1']))
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))

    trial.set_user_attr("err1", err_1)
    trial.set_user_attr("err2", err_2)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("logloss", logloss)

    return auc

# Clean up any active runs
mlflow.end_run()

# Setup experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Create parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Train final model with best params
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Create temporary directory for model artifacts
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Save model in both formats
        model_path = os.path.join(tmp_dir, "model")
        sklearn_model_path = os.path.join(tmp_dir, "sklearn_model")
        
        # Save as CatBoost native format
        best_model.save_model(os.path.join(model_path, "model.cbm"))
        
        # Create MLmodel file for CatBoost
        with open(os.path.join(model_path, "MLmodel"), 'w') as f:
            f.write(f"""
name: catboost
flavors:
  catboost:
    catboost_version: {catboost.__version__}
    data: model.cbm
""")
        
        # Log both versions
        mlflow.log_artifacts(model_path, "model")
        mlflow.sklearn.log_model(best_model, "sklearn_model")
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

/tmp/ipykernel_82920/2023761708.py:104: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:36:18,765] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:36:19,913] Trial 54 finished with value: 0.8251309803351858 and parameters: {'learning_rate': 0.030735013379106363, 'depth': 3, 'l2_leaf_reg': 4.226688892338806, 'random_strength': 1.0824666391262532}. Best is trial 45 with value: 0.8253653715190726.
[I 2025-08-12 23:36:20,947] Trial 55 finished with value: 0.8246605360613022 and parameters: {'learning_rate': 0.0309983005020658, 'depth': 2, 'l2_leaf_reg': 4.3406322435825135, 'random_strength': 1.0394180392679004}. Best is trial 45 with value: 0.8253653715190726.
[I 2025-08-12 23:36:22,352] Trial 56 finished with value: 0.8242491385228228 and parameters: {'learning_rate': 0.017702373503859253, 'depth': 4, 'l2_leaf_reg': 3.8814135936233

0:	learn: 0.6777118	total: 1.57ms	remaining: 1.57s
1:	learn: 0.6628989	total: 3.47ms	remaining: 1.73s
2:	learn: 0.6493109	total: 5ms	remaining: 1.66s
3:	learn: 0.6368717	total: 6.39ms	remaining: 1.59s
4:	learn: 0.6245517	total: 7.85ms	remaining: 1.56s
5:	learn: 0.6128568	total: 9.43ms	remaining: 1.56s
6:	learn: 0.6021530	total: 10.8ms	remaining: 1.54s
7:	learn: 0.5926540	total: 12.3ms	remaining: 1.52s
8:	learn: 0.5836529	total: 13.7ms	remaining: 1.51s
9:	learn: 0.5749220	total: 15.2ms	remaining: 1.5s
10:	learn: 0.5668232	total: 16.6ms	remaining: 1.5s
11:	learn: 0.5597039	total: 18ms	remaining: 1.48s
12:	learn: 0.5527957	total: 19.4ms	remaining: 1.48s
13:	learn: 0.5465670	total: 20.9ms	remaining: 1.47s
14:	learn: 0.5411488	total: 22.2ms	remaining: 1.46s
15:	learn: 0.5355972	total: 23.6ms	remaining: 1.45s
16:	learn: 0.5300792	total: 25.2ms	remaining: 1.46s
17:	learn: 0.5254251	total: 26.7ms	remaining: 1.46s
18:	learn: 0.5212166	total: 28.1ms	remaining: 1.45s
19:	learn: 0.5170139	total: 2

CatBoostError: (No such file or directory) /src/catboost/util/system/file.cpp:857: can't open "/tmp/tmpifxeprzw/model/model.cbm" with mode WrOnly|CreateAlways|Seq (0x00000034)

In [33]:
# ... (keep all previous imports and setup code)
import catboost
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Train final model with best params
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Create temporary directory for model artifacts
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Create model directory structure
        model_path = os.path.join(tmp_dir, "model")
        os.makedirs(model_path, exist_ok=True)
        
        # Save as CatBoost native format
        model_file = os.path.join(model_path, "model.cbm")
        best_model.save_model(model_file)
        
        # Create MLmodel file for CatBoost
        mlmodel_content = f"""
name: catboost
flavors:
  catboost:
    catboost_version: {catboost.__version__}
    data: model.cbm
"""
        with open(os.path.join(model_path, "MLmodel"), 'w') as f:
            f.write(mlmodel_content.strip())
        
        # Also save as sklearn pickle format
        sklearn_path = os.path.join(tmp_dir, "sklearn_model")
        os.makedirs(sklearn_path, exist_ok=True)
        
        import pickle
        with open(os.path.join(sklearn_path, "model.pkl"), 'wb') as f:
            pickle.dump(best_model, f)
        
        # Create MLmodel file for sklearn
        sklearn_mlmodel = """
name: sklearn
flavors:
  sklearn:
    sklearn_version: 1.0.2
    pickled_model: model.pkl
"""
        with open(os.path.join(sklearn_path, "MLmodel"), 'w') as f:
            f.write(sklearn_mlmodel.strip())
        
        # Log all artifacts
        mlflow.log_artifacts(tmp_dir)
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")


/tmp/ipykernel_82920/2009991330.py:7: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:39:24,141] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:39:25,180] Trial 74 finished with value: 0.8230699707501141 and parameters: {'learning_rate': 0.03447435118410182, 'depth': 4, 'l2_leaf_reg': 4.102713916620556, 'random_strength': 0.2495020766694027}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:39:26,437] Trial 75 finished with value: 0.8247056667707918 and parameters: {'learning_rate': 0.044940305860879305, 'depth': 3, 'l2_leaf_reg': 3.770795423099187, 'random_strength': 1.1145933093876415}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:39:27,573] Trial 76 finished with value: 0.8247692817703793 and parameters: {'learning_rate': 0.04158551442253223, 'depth': 2, 'l2_leaf_reg': 3.970427726317753,

0:	learn: 0.6769967	total: 1.58ms	remaining: 1.58s
1:	learn: 0.6617117	total: 3.19ms	remaining: 1.59s
2:	learn: 0.6468421	total: 4.56ms	remaining: 1.51s
3:	learn: 0.6333876	total: 5.87ms	remaining: 1.46s
4:	learn: 0.6221412	total: 7.17ms	remaining: 1.43s
5:	learn: 0.6114146	total: 8.46ms	remaining: 1.4s
6:	learn: 0.5998912	total: 9.82ms	remaining: 1.39s
7:	learn: 0.5896462	total: 11.2ms	remaining: 1.39s
8:	learn: 0.5806116	total: 12.5ms	remaining: 1.37s
9:	learn: 0.5717455	total: 14.9ms	remaining: 1.47s
10:	learn: 0.5636009	total: 16.5ms	remaining: 1.48s
11:	learn: 0.5563021	total: 18.1ms	remaining: 1.49s
12:	learn: 0.5496657	total: 19.7ms	remaining: 1.5s
13:	learn: 0.5431384	total: 21.3ms	remaining: 1.5s
14:	learn: 0.5371724	total: 22.9ms	remaining: 1.5s
15:	learn: 0.5312546	total: 24.5ms	remaining: 1.51s
16:	learn: 0.5257262	total: 27.4ms	remaining: 1.59s
17:	learn: 0.5211269	total: 29ms	remaining: 1.58s
18:	learn: 0.5168913	total: 30.6ms	remaining: 1.58s
19:	learn: 0.5128246	total: 

In [36]:
run_id

'21f9f600cd6946f0818a5c45a68171d5'

In [40]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os
import tempfile
import pickle
import catboost

# Environment setup
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)

    metrics = defaultdict(list)
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, probas)
        
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)

    err_1 = median(array(metrics['err1']))
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))

    trial.set_user_attr("err1", err_1)
    trial.set_user_attr("err2", err_2)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("logloss", logloss)

    return auc

# Clean up any active runs
mlflow.end_run()

# Setup experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Create parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    
    # Initialize callback
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={
            'nested': True,
            'tags': {MLFLOW_PARENT_RUN_ID: run_id}
        }
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    
    # Train final model with best params
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Save model using mlflow's built-in catboost flavor
    mlflow.catboost.log_model(
        cb_model=best_model,
        artifact_path="model",
        registered_model_name="best_catboost_model",
        conda_env={
            'channels': ['defaults'],
            'dependencies': [
                f'python',
                f'catboost=={catboost.__version__}',
                'mlflow',
                'scikit-learn'
            ]
        }
    )
    
    # Also save as sklearn flavor for compatibility
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="sklearn_model",
        conda_env={
            'channels': ['defaults'],
            'dependencies': [
                f'python',
                f'catboost=={catboost.__version__}',
                'mlflow',
                'scikit-learn'
            ]
        }
    )
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")
print(f"Run ID: {run_id}")

/tmp/ipykernel_82920/300382568.py:106: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:48:43,224] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-08-12 23:48:44,343] Trial 104 finished with value: 0.8249850682838631 and parameters: {'learning_rate': 0.050941048357995254, 'depth': 2, 'l2_leaf_reg': 4.411637466009875, 'random_strength': 1.41906682024925}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:48:45,760] Trial 105 finished with value: 0.825150967068609 and parameters: {'learning_rate': 0.05666531453325653, 'depth': 2, 'l2_leaf_reg': 4.650036247489821, 'random_strength': 1.3495976485242336}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:48:47,892] Trial 106 finished with value: 0.8190429386895566 and parameters: {'learning_rate': 0.03879811266680684, 'depth': 1, 'l2_leaf_reg': 4.465468598553427

0:	learn: 0.6769967	total: 7.49ms	remaining: 7.48s
1:	learn: 0.6617117	total: 11.2ms	remaining: 5.57s
2:	learn: 0.6468421	total: 16.2ms	remaining: 5.4s
3:	learn: 0.6333876	total: 20.4ms	remaining: 5.07s
4:	learn: 0.6221412	total: 28.1ms	remaining: 5.58s
5:	learn: 0.6114146	total: 31.3ms	remaining: 5.19s
6:	learn: 0.5998912	total: 34.6ms	remaining: 4.9s
7:	learn: 0.5896462	total: 38.4ms	remaining: 4.76s
8:	learn: 0.5806116	total: 42.3ms	remaining: 4.66s
9:	learn: 0.5717455	total: 45.5ms	remaining: 4.51s
10:	learn: 0.5636009	total: 49.2ms	remaining: 4.42s
11:	learn: 0.5563021	total: 52.5ms	remaining: 4.33s
12:	learn: 0.5496657	total: 56.1ms	remaining: 4.26s
13:	learn: 0.5431384	total: 61.2ms	remaining: 4.31s
14:	learn: 0.5371724	total: 64ms	remaining: 4.2s
15:	learn: 0.5312546	total: 67.5ms	remaining: 4.15s
16:	learn: 0.5257262	total: 71.1ms	remaining: 4.11s
17:	learn: 0.5211269	total: 74.5ms	remaining: 4.07s
18:	learn: 0.5168913	total: 77.9ms	remaining: 4.02s
19:	learn: 0.5128246	total:

Registered model 'best_catboost_model' already exists. Creating a new version of this model...
2025/08/12 23:49:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_catboost_model, version 3
Created version '3' of model 'best_catboost_model'.


Number of finished trials: 114
Best params: {'learning_rate': 0.03371574251464615, 'depth': 3, 'l2_leaf_reg': 3.7036324067522632, 'random_strength': 0.6126207400368813}
Run ID: 3a52748a8b8247db92047f4a4ea87e3d


In [42]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os
import catboost
import boto3
from mlflow.tracking import MlflowClient

# ====================== CONFIGURATION ======================
# Environment setup
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

# Server configuration
TRACKING_SERVER_HOST = "localhost"  # replace with your host
TRACKING_SERVER_PORT = "5000"       # replace with your port

# Experiment configuration
EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

# ====================== INITIAL SETUP ======================
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# Verify S3 connection
try:
    s3 = boto3.client('s3',
                     endpoint_url=os.getenv("MLFLOW_S3_ENDPOINT_URL"),
                     aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
                     aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"))

    print("✅ S3 connection verified successfully")
except Exception as e:
    print(f"❌ S3 connection failed: {str(e)}")
    raise

# ====================== OBJECTIVE FUNCTION ======================
def objective(trial: optuna.Trial) -> float:
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    
    model = CatBoostClassifier(**params)
    skf = StratifiedKFold(n_splits=2)
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction))
        metrics["recall"].append(recall_score(val_y, prediction))
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1_score(val_y, prediction))
        metrics["logloss"].append(log_loss(val_y, probas))

    trial.set_user_attr("err1", median(array(metrics['err1'])))
    trial.set_user_attr("err2", median(array(metrics['err2'])))
    trial.set_user_attr("precision", median(array(metrics['precision'])))
    trial.set_user_attr("recall", median(array(metrics['recall'])))
    trial.set_user_attr("f1", median(array(metrics['f1'])))
    trial.set_user_attr("logloss", median(array(metrics['logloss'])))

    return median(array(metrics['auc']))

# ====================== MAIN EXECUTION ======================
# Clean up any active runs
mlflow.end_run()

# Setup experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    print(f"Created new experiment with ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Using existing experiment with ID: {experiment_id}")

# Create parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    print(f"Started parent run with ID: {run_id}")
    
    # Initialize MLflow callback
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={'nested': True, 'tags': {MLFLOW_PARENT_RUN_ID: run_id}}
    )
    
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    # Run optimization
    print("Starting optimization...")
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    print("Optimization completed!")
    
    # Train final model with best params
    best_params = study.best_params
    print(f"Best parameters: {best_params}")
    
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Log CatBoost model
    print("Logging CatBoost model...")
    cb_model_info = mlflow.catboost.log_model(
        cb_model=best_model,
        artifact_path="catboost_model",
        registered_model_name="best_catboost_model",
        conda_env={
            'channels': ['defaults'],
            'dependencies': [
                f'python',
                f'catboost=={catboost.__version__}',
                'mlflow',
                'scikit-learn'
            ]
        }
    )
    print(f"✅ CatBoost model logged to: {cb_model_info.model_uri}")
    
    # Log sklearn model
    print("Logging sklearn model...")
    sklearn_model_info = mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="sklearn_model",
        conda_env={
            'channels': ['defaults'],
            'dependencies': [
                f'python',
                f'catboost=={catboost.__version__}',
                'mlflow',
                'scikit-learn'
            ]
        }
    )
    print(f"✅ Sklearn model logged to: {sklearn_model_info.model_uri}")
    
    # Log parameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })
    
    # Verify artifacts
    client = MlflowClient()
    print("\nLogged artifacts:")
    for artifact in client.list_artifacts(run_id):
        print(f" - {artifact.path}")

# Final results
print("\n=== RESULTS ===")
print(f"Best AUC: {study.best_value:.4f}")
print(f"Best params: {best_params}")
print(f"Run ID: {run_id}")
print(f"CatBoost model URI: {cb_model_info.model_uri}")
print(f"Sklearn model URI: {sklearn_model_info.model_uri}")

✅ S3 connection verified successfully
Using existing experiment with ID: 16
Started parent run with ID: 1ccd3b0b016e41349ee48d7abb2303fd


/tmp/ipykernel_82920/1935404882.py:115: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:56:40,984] Using an existing study with name 'churn_model' instead of creating a new one.


Starting optimization...


[I 2025-08-12 23:56:43,125] Trial 114 finished with value: 0.8250949089289612 and parameters: {'learning_rate': 0.058735867844103784, 'depth': 2, 'l2_leaf_reg': 4.6736056846203, 'random_strength': 0.8416792525164823}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:56:44,138] Trial 115 finished with value: 0.8208815528275752 and parameters: {'learning_rate': 0.04694653413771434, 'depth': 1, 'l2_leaf_reg': 4.46367386125311, 'random_strength': 1.3435839094238806}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:56:45,280] Trial 116 finished with value: 0.8248211461014318 and parameters: {'learning_rate': 0.03746059419614123, 'depth': 3, 'l2_leaf_reg': 4.933283343510124, 'random_strength': 1.3972866750895396}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:56:47,015] Trial 117 finished with value: 0.8244401700836458 and parameters: {'learning_rate': 0.051117303696494774, 'depth': 2, 'l2_leaf_reg': 4.813365693351874, 'random_strength':

Optimization completed!
Best parameters: {'learning_rate': 0.03371574251464615, 'depth': 3, 'l2_leaf_reg': 3.7036324067522632, 'random_strength': 0.6126207400368813}
0:	learn: 0.6769967	total: 1.48ms	remaining: 1.48s
1:	learn: 0.6617117	total: 2.84ms	remaining: 1.42s
2:	learn: 0.6468421	total: 4.21ms	remaining: 1.4s
3:	learn: 0.6333876	total: 5.51ms	remaining: 1.37s
4:	learn: 0.6221412	total: 6.85ms	remaining: 1.36s
5:	learn: 0.6114146	total: 8.38ms	remaining: 1.39s
6:	learn: 0.5998912	total: 9.76ms	remaining: 1.38s
7:	learn: 0.5896462	total: 11.1ms	remaining: 1.38s
8:	learn: 0.5806116	total: 12.5ms	remaining: 1.37s
9:	learn: 0.5717455	total: 13.8ms	remaining: 1.37s
10:	learn: 0.5636009	total: 15.2ms	remaining: 1.37s
11:	learn: 0.5563021	total: 16.6ms	remaining: 1.37s
12:	learn: 0.5496657	total: 18ms	remaining: 1.36s
13:	learn: 0.5431384	total: 19.4ms	remaining: 1.36s
14:	learn: 0.5371724	total: 20.8ms	remaining: 1.36s
15:	learn: 0.5312546	total: 23ms	remaining: 1.41s
16:	learn: 0.5257

Registered model 'best_catboost_model' already exists. Creating a new version of this model...
2025/08/12 23:56:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_catboost_model, version 4
Created version '4' of model 'best_catboost_model'.


✅ CatBoost model logged to: runs:/1ccd3b0b016e41349ee48d7abb2303fd/catboost_model
Logging sklearn model...
✅ Sklearn model logged to: runs:/1ccd3b0b016e41349ee48d7abb2303fd/sklearn_model

Logged artifacts:
 - catboost_model
 - sklearn_model

=== RESULTS ===
Best AUC: 0.8255
Best params: {'learning_rate': 0.03371574251464615, 'depth': 3, 'l2_leaf_reg': 3.7036324067522632, 'random_strength': 0.6126207400368813}
Run ID: 1ccd3b0b016e41349ee48d7abb2303fd
CatBoost model URI: runs:/1ccd3b0b016e41349ee48d7abb2303fd/catboost_model
Sklearn model URI: runs:/1ccd3b0b016e41349ee48d7abb2303fd/sklearn_model


In [43]:
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from numpy import array, median
from collections import defaultdict
from sklearn.metrics import (confusion_matrix, roc_auc_score, 
                           precision_score, recall_score, 
                           f1_score, log_loss)
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os
import catboost
from mlflow.tracking import MlflowClient

# ====================== CONFIGURATION ======================
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "localhost"
TRACKING_SERVER_PORT = "5000"

EXPERIMENT_NAME = "catboost_hyperopt"
RUN_NAME = "model_bayesian_search"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

# ====================== INITIAL SETUP ======================
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# ====================== OBJECTIVE FUNCTION ======================
def objective(trial: optuna.Trial) -> float:
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    
    model = CatBoostClassifier(**params)
    skf = StratifiedKFold(n_splits=2)
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction))
        metrics["recall"].append(recall_score(val_y, prediction))
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["f1"].append(f1_score(val_y, prediction))
        metrics["logloss"].append(log_loss(val_y, probas))

    trial.set_user_attr("err1", median(array(metrics['err1'])))
    trial.set_user_attr("err2", median(array(metrics['err2'])))
    trial.set_user_attr("precision", median(array(metrics['precision'])))
    trial.set_user_attr("recall", median(array(metrics['recall'])))
    trial.set_user_attr("f1", median(array(metrics['f1'])))
    trial.set_user_attr("logloss", median(array(metrics['logloss'])))

    return median(array(metrics['auc']))

# ====================== MAIN EXECUTION ======================
mlflow.end_run()

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    print(f"Created new experiment with ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Using existing experiment with ID: {experiment_id}")

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    run_id = parent_run.info.run_id
    print(f"Started parent run with ID: {run_id}")
    
    mlflc = MLflowCallback(
        tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
        metric_name='AUC',
        create_experiment=False,
        mlflow_kwargs={'nested': True, 'tags': {MLFLOW_PARENT_RUN_ID: run_id}}
    )
    
    study = optuna.create_study(
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        sampler=optuna.samplers.TPESampler(),
        load_if_exists=True
    )
    
    print("Starting optimization...")
    study.optimize(objective, n_trials=10, callbacks=[mlflc])
    print("Optimization completed!")
    
    best_params = study.best_params
    print(f"Best parameters: {best_params}")
    
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    # Save model to temporary directory first
    import tempfile
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Save CatBoost model
        cb_model_path = os.path.join(tmp_dir, "catboost_model")
        best_model.save_model(os.path.join(cb_model_path, "model.cbm"))
        
        # Create MLmodel file for CatBoost
        with open(os.path.join(cb_model_path, "MLmodel"), 'w') as f:
            f.write(f"""
flavors:
  catboost:
    data: model.cbm
    cb_version: {catboost.__version__}
model_uuid: {run_id}
run_id: {run_id}
utc_time_created: '2025-08-12 23:56:58.000000'""")
        
        # Log the entire directory as artifact
        mlflow.log_artifacts(cb_model_path, "catboost_model")
    
    # Similarly for sklearn model
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Save sklearn model
        sklearn_model_path = os.path.join(tmp_dir, "sklearn_model")
        with open(os.path.join(sklearn_model_path, "model.pkl"), 'wb') as f:
            pickle.dump(best_model, f)
        
        # Create MLmodel file for sklearn
        with open(os.path.join(sklearn_model_path, "MLmodel"), 'w') as f:
            f.write(f"""
flavors:
  sklearn:
    pickled_model: model.pkl
    sklearn_version: 1.0.2
    serialization_format: cloudpickle
model_uuid: {run_id}
run_id: {run_id}
utc_time_created: '2025-08-12 23:56:58.000000'""")
        
        mlflow.log_artifacts(sklearn_model_path, "sklearn_model")
    
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        'auc': study.best_value,
        'err1': study.best_trial.user_attrs['err1'],
        'err2': study.best_trial.user_attrs['err2'],
        'precision': study.best_trial.user_attrs['precision'],
        'recall': study.best_trial.user_attrs['recall'],
        'f1': study.best_trial.user_attrs['f1'],
        'logloss': study.best_trial.user_attrs['logloss']
    })

print("\n=== RESULTS ===")
print(f"Best AUC: {study.best_value:.4f}")
print(f"Best params: {best_params}")
print(f"Run ID: {run_id}")

/tmp/ipykernel_82920/394873740.py:95: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-08-12 23:59:28,937] Using an existing study with name 'churn_model' instead of creating a new one.


Using existing experiment with ID: 16
Started parent run with ID: 6de67a1661cd44e6a78ad643760d5f33
Starting optimization...


[I 2025-08-12 23:59:30,896] Trial 124 finished with value: 0.8248147579564633 and parameters: {'learning_rate': 0.05519656293638576, 'depth': 2, 'l2_leaf_reg': 4.060813401902627, 'random_strength': 1.8322769390158413}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:59:32,548] Trial 125 finished with value: 0.8224696672991381 and parameters: {'learning_rate': 0.06448141181337429, 'depth': 3, 'l2_leaf_reg': 3.9115026258530468, 'random_strength': 0.967423799428222}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:59:35,023] Trial 126 finished with value: 0.8249324785381167 and parameters: {'learning_rate': 0.04468731205886245, 'depth': 2, 'l2_leaf_reg': 3.776682774893099, 'random_strength': 1.6186951139671966}. Best is trial 72 with value: 0.8254518810982665.
[I 2025-08-12 23:59:36,809] Trial 127 finished with value: 0.8162807695145441 and parameters: {'learning_rate': 0.02881303632207669, 'depth': 1, 'l2_leaf_reg': 3.40890702883785, 'random_strength':

Optimization completed!
Best parameters: {'learning_rate': 0.032394252240081134, 'depth': 3, 'l2_leaf_reg': 4.191330329801244, 'random_strength': 1.4646578321312331}
0:	learn: 0.6842784	total: 1.27ms	remaining: 1.27s
1:	learn: 0.6697482	total: 2.63ms	remaining: 1.31s
2:	learn: 0.6559775	total: 4.11ms	remaining: 1.37s
3:	learn: 0.6437064	total: 6.23ms	remaining: 1.55s
4:	learn: 0.6321034	total: 7.45ms	remaining: 1.48s
5:	learn: 0.6206839	total: 8.54ms	remaining: 1.41s
6:	learn: 0.6103705	total: 9.72ms	remaining: 1.38s
7:	learn: 0.6025046	total: 10.8ms	remaining: 1.33s
8:	learn: 0.5927194	total: 11.9ms	remaining: 1.3s
9:	learn: 0.5844471	total: 13.3ms	remaining: 1.31s
10:	learn: 0.5757551	total: 14.8ms	remaining: 1.33s
11:	learn: 0.5671330	total: 16.3ms	remaining: 1.34s
12:	learn: 0.5598065	total: 17.9ms	remaining: 1.36s
13:	learn: 0.5550950	total: 19.3ms	remaining: 1.36s
14:	learn: 0.5488664	total: 20.8ms	remaining: 1.36s
15:	learn: 0.5426722	total: 22.2ms	remaining: 1.37s
16:	learn: 0.

CatBoostError: (No such file or directory) /src/catboost/util/system/file.cpp:857: can't open "/tmp/tmp86_usdv9/catboost_model/model.cbm" with mode WrOnly|CreateAlways|Seq (0x00000034)

In [44]:
from mlflow.tracking import MlflowClient
import mlflow
import os
import tempfile
import pickle

# Установите те же параметры подключения
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri("http://localhost:5000")  # Ваш tracking URI

# Укажите run_id из вашего предыдущего запуска
run_id = "1ccd3b0b016e41349ee48d7abb2303fd"
client = MlflowClient()

# 1. Пересохранение CatBoost модели
with tempfile.TemporaryDirectory() as tmp_dir:
    # Создаем структуру каталогов
    cb_dir = os.path.join(tmp_dir, "catboost_model")
    os.makedirs(cb_dir, exist_ok=True)
    
    # Загружаем модель из оригинального run
    model_path = client.download_artifacts(run_id, "catboost_model", tmp_dir)
    
    # Если нужно пересохранить саму модель (альтернативный вариант)
    # model = mlflow.catboost.load_model(f"runs:/{run_id}/catboost_model")
    # model.save_model(os.path.join(cb_dir, "model.cbm"))
    
    # Создаем MLmodel файл
    with open(os.path.join(cb_dir, "MLmodel"), 'w') as f:
        f.write(f"""
flavors:
  catboost:
    data: model.cbm
    cb_version: {catboost.__version__}
model_uuid: {run_id}
run_id: {run_id}
utc_time_created: '2025-08-12 23:56:58.000000'""")
    
    # Перезаписываем артефакты
    client.log_artifacts(run_id, cb_dir, "catboost_model")

# 2. Пересохранение Sklearn модели
with tempfile.TemporaryDirectory() as tmp_dir:
    sk_dir = os.path.join(tmp_dir, "sklearn_model")
    os.makedirs(sk_dir, exist_ok=True)
    
    # Загружаем и сохраняем модель
    model = mlflow.sklearn.load_model(f"runs:/{run_id}/sklearn_model")
    with open(os.path.join(sk_dir, "model.pkl"), 'wb') as f:
        pickle.dump(model, f)
    
    # Создаем MLmodel файл
    with open(os.path.join(sk_dir, "MLmodel"), 'w') as f:
        f.write(f"""
flavors:
  sklearn:
    pickled_model: model.pkl
    sklearn_version: 1.0.2
    serialization_format: cloudpickle
model_uuid: {run_id}
run_id: {run_id}
utc_time_created: '2025-08-12 23:56:58.000000'""")
    
    client.log_artifacts(run_id, sk_dir, "sklearn_model")

print("Модели успешно пересохранены!")

Модели успешно пересохранены!


In [45]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_id = "1ccd3b0b016e41349ee48d7abb2303fd"

# 1. Создаем каталог cv и копируем туда модели
with tempfile.TemporaryDirectory() as tmp_dir:
    # Скачиваем существующие модели
    cb_model_path = client.download_artifacts(run_id, "catboost_model", tmp_dir)
    sk_model_path = client.download_artifacts(run_id, "sklearn_model", tmp_dir)
    
    # Создаем структуру для cv
    cv_dir = os.path.join(tmp_dir, "cv")
    os.makedirs(cv_dir, exist_ok=True)
    
    # Копируем CatBoost модель
    os.rename(
        os.path.join(cb_model_path, "model.cbm"),
        os.path.join(cv_dir, "model.pkl")  # Переименовываем в model.pkl
    )
    
    # Копируем MLmodel файл
    os.rename(
        os.path.join(cb_model_path, "MLmodel"),
        os.path.join(cv_dir, "MLmodel")
    )
    
    # Загружаем обратно в S3
    client.log_artifacts(run_id, cv_dir, "cv")

print("Модели успешно скопированы в /cv каталог!")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpqtkxdsi9/catboost_model/model.cbm' -> '/tmp/tmpqtkxdsi9/cv/model.pkl'

In [46]:
from mlflow.tracking import MlflowClient
import os
import shutil
import tempfile

# Инициализация клиента
client = MlflowClient()
run_id = "1ccd3b0b016e41349ee48d7abb2303fd"

with tempfile.TemporaryDirectory() as tmp_dir:
    print(f"Рабочая директория: {tmp_dir}")
    
    # 1. Скачиваем все артефакты
    artifacts_path = client.download_artifacts(run_id, "", tmp_dir)
    print(f"Артефакты скачаны в: {artifacts_path}")
    
    # 2. Проверяем структуру скачанных файлов
    print("Содержимое артефактов:")
    for root, dirs, files in os.walk(artifacts_path):
        for file in files:
            print(f" - {os.path.join(root, file)}")
    
    # 3. Создаем каталог cv
    cv_dir = os.path.join(tmp_dir, "cv")
    os.makedirs(cv_dir, exist_ok=True)
    print(f"Создан каталог cv: {cv_dir}")
    
    # 4. Копируем нужные файлы
    try:
        # Ищем model.cbm или model.pkl
        model_files = []
        for root, dirs, files in os.walk(artifacts_path):
            for file in files:
                if file in ["model.cbm", "model.pkl"]:
                    model_files.append(os.path.join(root, file))
        
        if not model_files:
            raise FileNotFoundError("Не найдены файлы модели (model.cbm или model.pkl)")
        
        # Копируем первый найденный файл модели
        src_model = model_files[0]
        dst_model = os.path.join(cv_dir, "model.pkl")
        shutil.copy2(src_model, dst_model)
        print(f"Скопировано: {src_model} -> {dst_model}")
        
        # Ищем MLmodel файл
        mlmodel_files = []
        for root, dirs, files in os.walk(artifacts_path):
            for file in files:
                if file == "MLmodel":
                    mlmodel_files.append(os.path.join(root, file))
        
        if mlmodel_files:
            src_mlmodel = mlmodel_files[0]
            dst_mlmodel = os.path.join(cv_dir, "MLmodel")
            shutil.copy2(src_mlmodel, dst_mlmodel)
            print(f"Скопировано: {src_mlmodel} -> {dst_mlmodel}")
        else:
            print("Предупреждение: Не найден MLmodel файл")
        
        # 5. Загружаем обратно в MLflow
        client.log_artifacts(run_id, cv_dir, "cv")
        print("Файлы успешно загружены в каталог /cv")
        
    except Exception as e:
        print(f"Ошибка: {str(e)}")
        raise

Рабочая директория: /tmp/tmp21yva_vg


2025/08/13 00:03:03 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Артефакты скачаны в: /tmp/tmp21yva_vg/
Содержимое артефактов:
 - /tmp/tmp21yva_vg/catboost_model/model.cb
 - /tmp/tmp21yva_vg/catboost_model/python_env.yaml
 - /tmp/tmp21yva_vg/catboost_model/MLmodel
 - /tmp/tmp21yva_vg/catboost_model/conda.yaml
 - /tmp/tmp21yva_vg/catboost_model/requirements.txt
 - /tmp/tmp21yva_vg/sklearn_model/python_env.yaml
 - /tmp/tmp21yva_vg/sklearn_model/MLmodel
 - /tmp/tmp21yva_vg/sklearn_model/model.pkl
 - /tmp/tmp21yva_vg/sklearn_model/conda.yaml
 - /tmp/tmp21yva_vg/sklearn_model/requirements.txt
Создан каталог cv: /tmp/tmp21yva_vg/cv
Скопировано: /tmp/tmp21yva_vg/sklearn_model/model.pkl -> /tmp/tmp21yva_vg/cv/model.pkl
Скопировано: /tmp/tmp21yva_vg/catboost_model/MLmodel -> /tmp/tmp21yva_vg/cv/MLmodel
Файлы успешно загружены в каталог /cv
